In [381]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
from datetime import datetime
from datetime import timedelta




def dps_all(city,entity,start_date, end_date):

    query = """
    with details as (
    SELECT   smto.entity_id
            ,smto.country_code
            ,smto.city_id
            ,smto.city_name
            ,CONCAT(smto.city_name , "-", "post") city_name_var
            ,smto.created_date_local
            ,date_trunc(created_date_local, ISOWEEK) week
            ,date_trunc(created_date_local, MONTH) month
            ,smto.customer_account_id
            ,smto.customer_first_order_date
            ,smto.order_id
            ,smto.dps_delivery_fee_eur
            ,smto.delivery_fee_eur
            ,smto.delivery_fee_local
            ,dps_travel_time_fee_eur
            ,dps_standard_fee_eur
            ,vendor_funding_amount_local
            ,dps_last_non_zero_df_eur
            ,delivery_fee_vat_eur
            ,delivery_fee_vat_local
            ,smto.delivery_fee_option
            ,smto.priority_fee_eur
            ,smto.saver_discount_eur
            ,smto.gfv_eur
            ,smto.gfv_local
            ,smto.gmv_eur
            ,smto.gmv_local
            ,smto.profit_eur
            ,smto.profit_local
            ,smto.revenue_eur
            ,smto.revenue_local
            ,smto.mov_customer_fee_eur
            ,smto.service_fee_eur
            ,smto.dps_service_fee_eur
            ,smto.delivery_distance
            ,smto.delivery_costs_eur
            ,smto.delivery_costs_local
            ,smto.joker_vendor_fee_eur
            ,smto.is_sent
            ,smto.is_own_delivery
            ,smto.vertical_type
            ,(case when is_sent = TRUE then 1 else 0 end) completed_orders
            ,fully_loaded_gross_profit_eur
            ,smto.commission_eur
            ,smto.platform_order_code
            ,smto.has_new_customer_condition
            ,smto.new_customer_condition_type
    FROM `fulfillment-dwh-production.curated_data_shared.dps_sessions_mapped_to_orders` smto
    join
    (   SELECT  o.customer_account_id
        FROM `fulfillment-dwh-production.curated_data_shared.dps_sessions_mapped_to_orders` o
        WHERE o.entity_id = \"""" + entity + """\"
            AND o.created_date_local BETWEEN  \"""" + start_date + """\" AND \"""" + end_date + """\"
            and o.has_new_customer_condition is true
            and o.new_customer_condition_type = 'TOTAL'
            AND o.city_name in (""" + city + """)
            group by 1
    ) tm on tm.customer_account_id = smto.customer_account_id
    WHERE smto.created_date_local BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    AND city_name in (""" + city + """)
    AND is_own_delivery = TRUE
    AND is_sent = TRUE
    ),
    voucher_data as (
    SELECT  o.global_entity_id
            ,order_id
            ,o.customer_account_id
            ,vouchers[SAFE_OFFSET(0)].voucher_code voucher_code
            ,vouchers[SAFE_OFFSET(0)].amount_eur voucher_amount_eur
    FROM `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o
    where global_entity_id = \"""" + entity + """\"
    AND partition_date_local BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    )
    select d.*
           ,vd.voucher_amount_eur
    from details d 
    left join voucher_data vd on vd.global_entity_id = d.entity_id and vd.order_id = d.platform_order_code
    """

    return query

def voucher_all_v(city,entity,vouchers, start_date, end_date):

    query = """
    with details as (
    SELECT  smto.entity_id
            ,smto.country_code
            ,smto.city_id
            ,smto.city_name
            ,CONCAT(smto.city_name , "-", "post") city_name_var
            ,smto.created_date_local
            ,date_trunc(created_date_local, ISOWEEK) week
            ,date_trunc(created_date_local, MONTH) month
            ,smto.customer_account_id
            ,smto.customer_first_order_date
            ,smto.order_id
            ,smto.dps_delivery_fee_eur
            ,smto.delivery_fee_eur
            ,smto.delivery_fee_local
            ,dps_travel_time_fee_eur
            ,dps_standard_fee_eur
            ,vendor_funding_amount_local
            ,dps_last_non_zero_df_eur
            ,delivery_fee_vat_eur
            ,delivery_fee_vat_local
            ,smto.delivery_fee_option
            ,smto.priority_fee_eur
            ,smto.saver_discount_eur
            ,smto.gfv_eur
            ,smto.gfv_local
            ,smto.gmv_eur
            ,smto.gmv_local
            ,smto.profit_eur
            ,smto.profit_local
            ,smto.revenue_eur
            ,smto.revenue_local
            ,smto.mov_customer_fee_eur
            ,smto.service_fee_eur
            ,smto.dps_service_fee_eur
            ,smto.delivery_distance
            ,smto.delivery_costs_eur
            ,smto.delivery_costs_local
            ,smto.joker_vendor_fee_eur
            ,smto.is_sent
            ,smto.is_own_delivery
            ,smto.vertical_type
            ,(case when is_sent = TRUE then 1 else 0 end) completed_orders
            ,fully_loaded_gross_profit_eur
            ,smto.commission_eur
            ,smto.platform_order_code
            ,smto.has_new_customer_condition
            ,smto.new_customer_condition_type
    FROM `fulfillment-dwh-production.curated_data_shared.dps_sessions_mapped_to_orders` smto
    where smto.customer_account_id in 
    (
    SELECT  o.customer_account_id
    FROM `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o
    where global_entity_id =  \"""" + entity + """\" 
    AND partition_date_local BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    AND (vouchers[SAFE_OFFSET(0)].voucher_code in (""" + vouchers + """))
    --AND (vouchers[SAFE_OFFSET(0)].voucher_code in (""" + vouchers + """)
    --OR vouchers[SAFE_OFFSET(0)].voucher_code like 'mieletön%' 
    --OR vouchers[SAFE_OFFSET(0)].voucher_code like 'tajuton%'
    --)
    group by 1
    )
    AND smto.created_date_local BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    AND city_name in (""" + city + """)
    AND is_own_delivery = TRUE
    AND is_sent = TRUE
    ),
    voucher_data as (
    SELECT  o.global_entity_id
            ,order_id
            ,o.customer_account_id
            ,vouchers[SAFE_OFFSET(0)].voucher_code
            ,vouchers[SAFE_OFFSET(0)].amount_eur voucher_amount_eur
    FROM `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o
    where global_entity_id =  \"""" + entity + """\"
    AND partition_date_local BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    )
    select dd.*
          ,vd.voucher_amount_eur
    from details dd 
    left join voucher_data as vd on vd.global_entity_id = dd.entity_id and vd.order_id = dd.platform_order_code
    """

    return query


def sessions(entity, city, start_date, end_date):

    query = """
    SELECT count(distinct perseus_session_key) total_sessions,
       count(distinct sessions.perseus_client_id) total_users,
       count(distinct case when has_transaction = True then sessions.perseus_client_id else null end) total_orders,
       count(distinct case when has_transaction = True then sessions.perseus_client_id else null end) / count(distinct perseus_session_key)   
    FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_perseus_sessions` 
    WHERE created_date BETWEEN \"""" + start_date + """\" AND \"""" + end_date + """\"
    AND entity_id = \"""" + entity + """\" 
    and sessions.customer_status = 'New'
    and dps_zone[SAFE_OFFSET(0)].city_name in (""" + city + """)
    """
    
    return query

def analyze_city_orders(df, period):
    
   # Create a new column for the conditional forgone delivery fee
    df['foregone_delivery_fee'] = df.apply(
    lambda row: row['dps_standard_fee_eur'] if row['dps_delivery_fee_eur'] == 0 or pd.isnull(row['dps_delivery_fee_eur']) else 0,
    axis=1)

    df['entity_id_period'] = df['entity_id'] + '_' + period
        
    grouped_data_post = df.groupby('entity_id_period').agg({
        'platform_order_code': 'nunique',
        'customer_account_id': 'nunique',
        'gfv_eur': 'sum',
        'voucher_amount_eur': 'sum',
        'fully_loaded_gross_profit_eur': 'sum',
        'foregone_delivery_fee': 'sum'
    }).reset_index()

    grouped_data_post.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_data_post.columns]

    grouped_data_post = grouped_data_post.rename(columns={
        'platform_order_code': 'Orders',
        'customer_account_id': 'Acquistions',
        'voucher_amount_eur': 'Voucher_cost_eur',
        'gfv_eur':'GFV_eur',
        'fully_loaded_gross_profit_eur' : 'FLGP_eur',
        'foregone_delivery_fee': 'FDNC_cost_eur',
        })


    voucher_counts = df[(df['voucher_amount_eur'] > 0) & ((df['dps_delivery_fee_eur'] == 0) | (df['dps_delivery_fee_eur'].isna()))].groupby('entity_id_period')['platform_order_code'].nunique().reset_index()
    voucher_counts = voucher_counts.rename(columns={'platform_order_code': 'Stacked_orders'})

    grouped_data_post = grouped_data_post.merge(voucher_counts, on='entity_id_period', how='left').fillna(0)
    grouped_data_post[['GFV_eur','Voucher_cost_eur','FLGP_eur','FDNC_cost_eur']] = grouped_data_post[['GFV_eur','Voucher_cost_eur','FLGP_eur','FDNC_cost_eur']].astype(int)

    grouped_data_post['total_acquistion_cost'] = grouped_data_post['FDNC_cost_eur'] + grouped_data_post['Voucher_cost_eur']

    grouped_data_post = grouped_data_post[['entity_id_period','Orders','Stacked_orders','Acquistions','FDNC_cost_eur','Voucher_cost_eur','total_acquistion_cost','GFV_eur','FLGP_eur']]

    return grouped_data_post.round(0).set_index('entity_id_period').T

def cities_summary(pre_df, post_df):
    
    df2 = analyze_city_orders(pre_df, 'pre')
    df1 = analyze_city_orders(post_df, 'post')

    merged_df = pd.merge(df2, df1, left_index=True, right_index=True, how='outer')

    for col in merged_df.columns:
        if col.endswith('_post'):
            post = col
            pre = post.replace('_post', '_pre')  # Find the corresponding _pre column
            
            if pre in merged_df.columns:  # Ensure the _pre column exists
                # Calculate percentage change and store it in 'percentage_change'
                merged_df['percentage_change'] = round(
                    ((merged_df[post] - merged_df[pre]) / abs(merged_df[pre])) * 100, 0
                )
    
    return merged_df

def user_retention_wow(df):

    post_inter_df = df.copy()

    post_inter_df['first_order_date'] = post_inter_df.groupby('customer_account_id')['created_date_local'].transform('min')
    post_inter_df['week_diff'] = ((post_inter_df['created_date_local'] - post_inter_df['first_order_date']).dt.days // 7)

    total_users = post_inter_df['customer_account_id'].nunique()

    retention_df = post_inter_df.groupby('week_diff').agg(
        users_retained=('customer_account_id', 'nunique')
    ).reset_index()

    retention_df['wow_retention'] = round((retention_df['users_retained'] / retention_df['users_retained'][0]) * 100,2)
    
    return retention_df
  


# define a few things (project id, start date etc.)
project_id = "logistics-customer-staging"
client = bigquery.Client(project = project_id)

/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# SWEDEN NEW CUSTOMERS

In [384]:
## Treatment Cities ##

entity_id = 'OP_SE'

city = ('Stockholm')
city_str = ",".join([f"'{c}'" for c in city])

vouchers = ('nyburgare', 'mumsigt','megagott','mums150')
vouchers_str = ",".join([f"'{c}'" for c in vouchers])


## post time period ##
start_date_post = '2024-08-06'
end_date_post = '2024-09-25'
dps_post_treatment = client.query(dps_all_f(city_str, entity_id, start_date_post, end_date_post)).to_dataframe()

## pre time period ##

# Convert the strings to datetime objects
date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date_post, date_format)
end_date = datetime.strptime(end_date_post, date_format)

# Calculate the difference in days
days_difference = (end_date - start_date).days
days_difference

# Subtract the number of days from date1
new_date = start_date - timedelta(days=days_difference)
new_date_only = new_date.date()
new_date_string = new_date_only.strftime("%Y-%m-%d")

start_date_pre = new_date_string
end_date_pre = start_date_post
voucher_pre_treatment = client.query(voucher_all_v(city_str, entity_id, vouchers_str,start_date_pre, end_date_pre)).to_dataframe()

cities_summary(voucher_pre_treatment, dps_post_treatment)

entity_id_period
Acquistions
FDNC_cost_eur
FLGP_eur
GFV_eur
Orders
Stacked_orders
Voucher_cost_eur
total_acquistion_cost


In [300]:
print(user_retention_wow(dps_post_treatment))
print(user_retention_wow(voucher_pre_treatment))

   week_diff  users_retained  wow_retention
0          0           17530         100.00
1          1            1598           9.12
2          2            1292           7.37
3          3             990           5.65
4          4             764           4.36
5          5             491           2.80
6          6             261           1.49
7          7              45           0.26
   week_diff  users_retained  wow_retention
0          0            3689         100.00
1          1             135           3.66
2          2              57           1.55
3          3              49           1.33
4          4              51           1.38
5          5              33           0.89
6          6              16           0.43


In [301]:
post_sessions = client.query(sessions(entity_id, city_str, start_date_post, end_date_post)).to_dataframe()
pre_sessions = client.query(sessions(entity_id, city_str, start_date_pre, end_date_pre)).to_dataframe()

print(post_sessions , pre_sessions)

   total_sessions  total_users  total_orders       f0_
0          294255       186898         18180  0.061783    total_sessions  total_users  total_orders       f0_
0          123935        79816          9825  0.079275


In [302]:
## Control Cities ##

entity_id = 'OP_SE'

city = ('Umea','Malmo')
city_str = ",".join([f"'{c}'" for c in city])

vouchers = ('nyburgare', 'mumsigt','megagott','mums150')
vouchers_str = ",".join([f"'{c}'" for c in vouchers])

## post time period ##
start_date_post = '2024-08-06'
end_date_post = '2024-09-25'
voucher_post_control = client.query(voucher_all_v(city_str, entity_id, vouchers_str, start_date_post, end_date_post)).to_dataframe()


## pre time period ##

# Convert the strings to datetime objects
date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date_post, date_format)
end_date = datetime.strptime(end_date_post, date_format)

# Calculate the difference in days
days_difference = (end_date - start_date).days
days_difference

# Subtract the number of days from date1
new_date = start_date - timedelta(days=days_difference)
new_date_only = new_date.date()
new_date_string = new_date_only.strftime("%Y-%m-%d")

start_date_pre = new_date_string
end_date_pre = start_date_post
voucher_pre_control = client.query(voucher_all_v(city_str, entity_id, vouchers_str,start_date_pre, end_date_pre)).to_dataframe()

cities_summary(voucher_pre_control, voucher_post_control)

entity_id_period,OP_SE_pre,OP_SE_post,percentage_change
Acquistions,649,787,21.0
FDNC_cost_eur,1391,948,-32.0
FLGP_eur,-5516,-6048,-10.0
GFV_eur,18524,23857,29.0
Orders,794,933,18.0
Stacked_orders,234,174,-26.0
Voucher_cost_eur,8430,10393,23.0
total_acquistion_cost,9821,11341,15.0


In [303]:
print(user_retention_wow(voucher_post_control))
print(user_retention_wow(voucher_pre_control))

   week_diff  users_retained  wow_retention
0          0             787         100.00
1          1              27           3.43
2          2              22           2.80
3          3              12           1.52
4          4              12           1.52
5          5               7           0.89
6          6               4           0.51
   week_diff  users_retained  wow_retention
0          0             649         100.00
1          1              23           3.54
2          2              14           2.16
3          3              16           2.47
4          4              12           1.85
5          5               6           0.92
6          6               2           0.31
7          7               1           0.15


In [304]:
post_sessions = client.query(sessions(entity_id, city_str, start_date_post, end_date_post)).to_dataframe()
pre_sessions = client.query(sessions(entity_id, city_str, start_date_pre, end_date_pre)).to_dataframe()

print(post_sessions , pre_sessions)

   total_sessions  total_users  total_orders       f0_
0           32438        22023          2316  0.071398    total_sessions  total_users  total_orders       f0_
0           27087        17952          2459  0.090782


# FINLAND NEW CUSTOMERS

In [322]:
## Treatment Cities ##

entity_id_f = 'PO_FI'

city = ('Helsinki','Oulu')
city_str = ",".join([f"'{c}'" for c in city])

vouchers = ('mieletön', 'tajuton','uskomaton','arena','huikea')
vouchers_str = ",".join([f"'{c}'" for c in vouchers])

## post time period ##
start_date_post = '2024-08-16'
end_date_post = '2024-09-15'
dps_post = client.query(dps_all(city_str, entity_id, start_date_post, end_date_post)).to_dataframe()


## pre time period ##

# Convert the strings to datetime objects
date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date_post, date_format)
end_date = datetime.strptime(end_date_post, date_format)

# Calculate the difference in days
days_difference = (end_date - start_date).days
days_difference

# Subtract the number of days from date1
new_date = start_date - timedelta(days=days_difference)
new_date_only = new_date.date()
new_date_string = new_date_only.strftime("%Y-%m-%d")

start_date_pre = new_date_string
end_date_pre = start_date_post
voucher_pre = client.query(voucher_all_v(city_str, entity_id,vouchers_str,start_date_pre, end_date_pre)).to_dataframe()

cities_summary(voucher_pre, dps_post)

entity_id_period,PO_FI_pre,PO_FI_post,percentage_change
Acquistions,1897,4664,146.0
FDNC_cost_eur,867,16878,1847.0
FLGP_eur,-26189,-34837,-33.0
GFV_eur,58762,147435,151.0
Orders,2462,6104,148.0
Stacked_orders,305,2499,719.0
Voucher_cost_eur,27505,34034,24.0
total_acquistion_cost,28372,50912,79.0


In [323]:
print(user_retention_wow(dps_post))
print(user_retention_wow(voucher_pre))

   week_diff  users_retained  wow_retention
0          0            4664         100.00
1          1             304           6.52
2          2             194           4.16
3          3              94           2.02
4          4              20           0.43
   week_diff  users_retained  wow_retention
0          0            1897         100.00
1          1             109           5.75
2          2              76           4.01
3          3              35           1.85
4          4               7           0.37


In [324]:
post_sessions = client.query(sessions(entity_id, city_str, start_date_post, end_date_post)).to_dataframe()
pre_sessions = client.query(sessions(entity_id, city_str, start_date_pre, end_date_pre)).to_dataframe()

print(post_sessions , pre_sessions)

   total_sessions  total_users  total_orders       f0_
0           49868        35198          1923  0.038562    total_sessions  total_users  total_orders       f0_
0            5765         4457           168  0.029141


In [325]:
## Control Cities ##

entity_id = 'PO_FI'

city = ('Tampere','Turku')
city_str = ",".join([f"'{c}'" for c in city])

vouchers = ('mieletön', 'tajuton','uskomaton','arena','huikea')
vouchers_str = ",".join([f"'{c}'" for c in vouchers])

## post time period ##
start_date_post = '2024-08-16'
end_date_post = '2024-09-15'
voucher_post_control = client.query(voucher_all_v(city_str, entity_id, vouchers_str, start_date_post, end_date_post)).to_dataframe()

## pre time period ##

# Convert the strings to datetime objects
date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date_post, date_format)
end_date = datetime.strptime(end_date_post, date_format)

# Calculate the difference in days
days_difference = (end_date - start_date).days
days_difference

# Subtract the number of days from date1
new_date = start_date - timedelta(days=days_difference)
new_date_only = new_date.date()
new_date_string = new_date_only.strftime("%Y-%m-%d")

start_date_pre = new_date_string
end_date_pre = start_date_post
voucher_pre_control = client.query(voucher_all_v(city_str, entity_id, vouchers_str,start_date_pre, end_date_pre)).to_dataframe()

cities_summary(voucher_pre_control, voucher_post_control)

entity_id_period,PO_FI_pre,PO_FI_post,percentage_change
Acquistions,580,330,-43.0
FDNC_cost_eur,196,216,10.0
FLGP_eur,-7110,-4701,34.0
GFV_eur,18023,9550,-47.0
Orders,745,428,-43.0
Stacked_orders,49,57,16.0
Voucher_cost_eur,8380,5080,-39.0
total_acquistion_cost,8576,5296,-38.0


In [326]:
print(user_retention_wow(voucher_post_control))
print(user_retention_wow(voucher_pre_control))

   week_diff  users_retained  wow_retention
0          0             330         100.00
1          1              23           6.97
2          2              14           4.24
3          3               6           1.82
4          4               1           0.30
   week_diff  users_retained  wow_retention
0          0             580         100.00
1          1              32           5.52
2          2              19           3.28
3          3               8           1.38
4          4               2           0.34


In [327]:
post_sessions = client.query(sessions(entity_id, city_str, start_date_post, end_date_post)).to_dataframe()
pre_sessions = client.query(sessions(entity_id, city_str, start_date_pre, end_date_pre)).to_dataframe()

print(post_sessions , pre_sessions)

   total_sessions  total_users  total_orders  f0_
0             179          143             0  0.0    total_sessions  total_users  total_orders  f0_
0               6            6             0  0.0


In [352]:
start_date_pre

'2024-07-17'

In [335]:
tt = """
SELECT perseus_session_key total_sessions,
       sessions.perseus_client_id total_users,
       events[SAFE_OFFSET(0)].transaction_id
FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_perseus_sessions` 
WHERE created_date BETWEEN '2024-08-16' and '2024-09-15'
AND entity_id = 'PO_FI'
AND events[SAFE_OFFSET(0)].transaction_id is not null
group by 1,2,3

"""

test = client.query(tt).to_dataframe()


In [350]:
test.columns

Index(['total_sessions', 'total_users', 'transaction_id'], dtype='object')

In [348]:
a = test.merge(voucher_pre_control, left_on = 'entity_id', right_on='entity_id', how = 'left')
a

KeyError: 'entity_id'

In [344]:
a.isna().sum()

total_sessions                      0
total_users                      1237
transaction_id                      0
entity_id                        3015
country_code                     3015
city_id                          3015
city_name                        3015
city_name_var                    3015
created_date_local               3015
week                             3015
month                            3015
customer_account_id              3015
customer_first_order_date        3016
order_id                         3015
dps_delivery_fee_eur             3015
delivery_fee_eur                 3015
delivery_fee_local               3015
dps_travel_time_fee_eur          3015
dps_standard_fee_eur             3015
vendor_funding_amount_local      3016
dps_last_non_zero_df_eur         3015
delivery_fee_vat_eur             3015
delivery_fee_vat_local           3015
delivery_fee_option              3015
priority_fee_eur                 3015
saver_discount_eur               3016
gfv_eur     

In [375]:
def test():

        query = """
        with details as (
        SELECT   smto.entity_id
                ,smto.country_code
                ,smto.city_id
                ,smto.city_name
                ,CONCAT(smto.city_name , "-", "post") city_name_var
                ,smto.created_date_local
                ,date_trunc(created_date_local, ISOWEEK) week
                ,date_trunc(created_date_local, MONTH) month
                ,smto.customer_account_id
                ,smto.customer_first_order_date
                ,smto.order_id
                ,smto.dps_delivery_fee_eur
                ,smto.delivery_fee_eur
                ,smto.delivery_fee_local
                ,dps_travel_time_fee_eur
                ,dps_standard_fee_eur
                ,vendor_funding_amount_local
                ,dps_last_non_zero_df_eur
                ,delivery_fee_vat_eur
                ,delivery_fee_vat_local
                ,smto.delivery_fee_option
                ,smto.priority_fee_eur
                ,smto.saver_discount_eur
                ,smto.gfv_eur
                ,smto.gfv_local
                ,smto.gmv_eur
                ,smto.gmv_local
                ,smto.profit_eur
                ,smto.profit_local
                ,smto.revenue_eur
                ,smto.revenue_local
                ,smto.mov_customer_fee_eur
                ,smto.service_fee_eur
                ,smto.dps_service_fee_eur
                ,smto.delivery_distance
                ,smto.delivery_costs_eur
                ,smto.delivery_costs_local
                ,smto.joker_vendor_fee_eur
                ,smto.is_sent
                ,smto.is_own_delivery
                ,smto.vertical_type
                ,(case when is_sent = TRUE then 1 else 0 end) completed_orders
                ,fully_loaded_gross_profit_eur
                ,smto.commission_eur
                ,smto.platform_order_code
                ,smto.has_new_customer_condition
                ,smto.new_customer_condition_type
        FROM `fulfillment-dwh-production.curated_data_shared.dps_sessions_mapped_to_orders` smto
        join
        (   SELECT  o.customer_account_id
                FROM `fulfillment-dwh-production.curated_data_shared.dps_sessions_mapped_to_orders` o
                WHERE o.entity_id = 'OP_SE'
                --AND (date(o.customer_first_order_date) BETWEEN  '2024-08-06' AND '2024-09-25' 
                AND (o.customer_first_order_date is null
                OR date(o.customer_first_order_date) = '0001-01-01')
                AND o.city_name in ('Uppsala','Vasteras','Orebro','Stockholm')
                group by 1
        ) tm on tm.customer_account_id = smto.customer_account_id
        WHERE smto.created_date_local BETWEEN '2024-08-06' AND '2024-09-25'
        AND city_name in ('Uppsala','Vasteras','Orebro','Stockholm')
        AND is_own_delivery = TRUE
        AND is_sent = TRUE
        ),
        voucher_data as (
        SELECT  o.global_entity_id
                ,order_id
                ,o.customer_account_id
                ,vouchers[SAFE_OFFSET(0)].voucher_code voucher_code
                ,vouchers[SAFE_OFFSET(0)].amount_eur voucher_amount_eur
        FROM `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o
        where global_entity_id = 'OP_SE'
        AND partition_date_local BETWEEN '2024-08-06' AND '2024-09-25'
        )
        select d.*
                ,vd.voucher_amount_eur
        from details d 
        left join voucher_data vd on vd.global_entity_id = d.entity_id and vd.order_id = d.platform_order_code
        """

        return query

In [376]:
a = client.query(test()).to_dataframe()


In [378]:
a

,entity_id,country_code,city_id,city_name,city_name_var,created_date_local,week,month,customer_account_id,customer_first_order_date,...,is_sent,is_own_delivery,vertical_type,completed_orders,fully_loaded_gross_profit_eur,commission_eur,platform_order_code,has_new_customer_condition,new_customer_condition_type,voucher_amount_eur
0,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opy8bq5r,None,...,True,True,restaurants,1,-2.549214,1.870000000,m0tz-2435-di97,False,None,NaN
1,OP_SE,se,4,Uppsala,Uppsala-post,2024-08-28,2024-08-26,2024-08-01,opaze0dc,None,...,True,True,supermarket,1,3.010247,6.660000000,kcha-2435-fmzm,False,None,NaN
2,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opds2rg0,None,...,True,True,supermarket,1,6.432845,10.700000000,jx0p-2435-ilpd,False,None,NaN
3,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opimp2xd,None,...,True,True,supermarket,1,1.277548,6.600000000,ljek-2435-z1bv,False,None,NaN
4,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opkwd9gb,None,...,True,True,restaurants,1,-0.002256,3.830000000,o8gi-2435-o0zg,False,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782586,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,ope0uj2x,2021-04-25 15:52:27+00:00,...,True,True,restaurants,1,-0.625401,3.660000000,g4oq-2437-gi56,False,None,NaN
782587,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opq4hwlq,2020-02-16 11:46:54+00:00,...,True,True,restaurants,1,-0.117069,3.970000000,euyb-2437-l72u,False,None,NaN
782588,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opvoi2yr,2018-08-31 13:48:20+00:00,...,True,True,restaurants,1,0.400293,6.090000000,euyb-2437-apd3,False,None,NaN
782589,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opn0d0b2,2021-04-01 19:05:54+00:00,...,True,True,restaurants,1,7.492788,5.300000000,tdh5-2437-r7v9,False,None,NaN


In [377]:
a[(a['has_new_customer_condition'] == True) & (a['new_customer_condition_type'] == 'TOTAL')]

,entity_id,country_code,city_id,city_name,city_name_var,created_date_local,week,month,customer_account_id,customer_first_order_date,...,is_sent,is_own_delivery,vertical_type,completed_orders,fully_loaded_gross_profit_eur,commission_eur,platform_order_code,has_new_customer_condition,new_customer_condition_type,voucher_amount_eur
324,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opy9ydia,None,...,True,True,restaurants,1,-2.350022,2.100000000,ak9z-87wy,True,TOTAL,NaN
325,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opdabw7y,None,...,True,True,restaurants,1,-14.141695,4.330000000,j2ge-2435-j96i,True,TOTAL,13.220635
326,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opml2050,None,...,True,True,restaurants,1,3.868962,7.730000000,muz1-2435-p861,True,TOTAL,NaN
327,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,optkf0nz,None,...,True,True,restaurants,1,-1.987019,3.410000000,j2ge-2435-4tqf,True,TOTAL,2.644127
328,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-28,2024-08-26,2024-08-01,opjih8dq,None,...,True,True,restaurants,1,-2.170146,3.310000000,h5e9-2435-fmsj,True,TOTAL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764862,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,op8smifk,0001-01-01 00:00:00+00:00,...,True,True,restaurants,1,-3.079482,1.610000000,s8ph-2437-iwz6,True,TOTAL,NaN
764863,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opfic6lb,None,...,True,True,restaurants,1,6.048391,9.930000000,s6zq-2437-tvsq,True,TOTAL,NaN
764864,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opeky6z0,None,...,True,True,restaurants,1,4.472991,12.440000000,k5ac-2437-6wuo,True,TOTAL,NaN
764865,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,op9fukzj,None,...,True,True,restaurants,1,-7.815959,8.810000000,ox4y-2437-oakp,True,TOTAL,13.245290


In [371]:
dps_post_treatment[(dps_post_treatment['has_new_customer_condition'] == True) & (dps_post_treatment['new_customer_condition_type'] == 'TOTAL')]

,entity_id,country_code,city_id,city_name,city_name_var,created_date_local,week,month,customer_account_id,customer_first_order_date,...,vertical_type,completed_orders,fully_loaded_gross_profit_eur,commission_eur,platform_order_code,has_new_customer_condition,new_customer_condition_type,voucher_amount_eur,foregone_delivery_fee,entity_id_period
3,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,opd4vjjr,None,...,restaurants,1,-4.767724,1.320000000,ir4c-1z9p,True,TOTAL,NaN,1.672000000,OP_SE_post
4,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,op5b3il5,None,...,restaurants,1,-10.202068,7.130000000,yvj1-2438-vtnn,True,TOTAL,13.202203,1.672000000,OP_SE_post
5,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,opfxol98,None,...,restaurants,1,0.519114,3.290000000,k43b-2438-y057,True,TOTAL,NaN,2.552000000,OP_SE_post
6,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,oph74nkc,None,...,restaurants,1,1.578941,5.360000000,wwx8-2438-dlsb,True,TOTAL,NaN,4.313000000,OP_SE_post
7,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,opwf7nqs,None,...,restaurants,1,-13.533902,2.710000000,z0rn-2438-sqrt,True,TOTAL,11.441909,2.552000000,OP_SE_post
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28855,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,op8smifk,0001-01-01 00:00:00+00:00,...,restaurants,1,-3.079482,1.610000000,s8ph-2437-iwz6,True,TOTAL,NaN,5.210000000,OP_SE_post
28856,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opfic6lb,None,...,restaurants,1,6.048391,9.930000000,s6zq-2437-tvsq,True,TOTAL,NaN,1.678000000,OP_SE_post
28857,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opeky6z0,None,...,restaurants,1,4.472991,12.440000000,k5ac-2437-6wuo,True,TOTAL,NaN,1.678000000,OP_SE_post
28858,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,op9fukzj,None,...,restaurants,1,-7.815959,8.810000000,ox4y-2437-oakp,True,TOTAL,13.245290,1.678000000,OP_SE_post


In [380]:
dps_post_treatment[dps_post_treatment['customer_account_id'] == 'opy0n2gs']

,entity_id,country_code,city_id,city_name,city_name_var,created_date_local,week,month,customer_account_id,customer_first_order_date,...,vertical_type,completed_orders,fully_loaded_gross_profit_eur,commission_eur,platform_order_code,has_new_customer_condition,new_customer_condition_type,voucher_amount_eur,foregone_delivery_fee,entity_id_period
0,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-21,2024-09-16,2024-09-01,opy0n2gs,2024-08-25 15:34:53+00:00,...,restaurants,1,-1.307412,3.060000000,s1jq-2438-fmem,False,None,NaN,3.433000000,OP_SE_post
18958,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-14,2024-09-09,2024-09-01,opy0n2gs,2024-08-25 15:34:53+00:00,...,restaurants,1,4.495339,4.890000000,s4we-2437-e3uo,False,None,NaN,0,OP_SE_post
24274,OP_SE,se,10,Stockholm,Stockholm-post,2024-08-25,2024-08-19,2024-08-01,opy0n2gs,None,...,restaurants,1,1.823648,6.310000000,s4we-2434-a64d,True,TOTAL,NaN,3.417000000,OP_SE_post
28970,OP_SE,se,10,Stockholm,Stockholm-post,2024-09-15,2024-09-09,2024-09-01,opy0n2gs,2024-08-25 15:34:53+00:00,...,restaurants,1,-1.390106,1.760000000,snas-2437-iaes,False,None,NaN,0,OP_SE_post
